In [3]:
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import math

In [16]:
def mfcc(song,n=20):
    return librosa.feature.mfcc(y=song,n_mfcc=n)

def mel_specto(song):
    return librosa.feature.melspectrogram(y=song)

In [81]:
def vert_dct_log(DB, n=10):
    time = DB.shape[1]
    ans=np.zeros((n,time), dtype=float, order='C')
    for x in range(0,time):
        col = DB[:,x]
        ans[:,x] = (scipy.fft.dct(col)[0:n])
    return ans

def adjusted_dct(data, x_val, period, x_min = 0, n = 20):
    #for data at integers 0,1,2...N-1, xval should be 1/2,3/2,5/3..(2N-1)/2, and period should be 2N
    size = data.shape[0]
    x_max = x_min+period/2
    out = np.zeros(n, dtype=float, order='C')
    if size != x_val.shape[0]:
        print ('DCT sizes incompatible')
        return None

    #create dx
    dx = np.zeros(size, dtype=float, order='C')
    dx[0] = (x_val[2]+x_val[1])/2 - x_min
    dx[size-1] = x_max - (x_val[size-1]+x_val[size-2])/2
    for i in range(1,size-1):
        dx[i] = (x_val[i+1]+x_val[i])/2 - (x_val[i]+x_val[i-1])/2
    
    for i in range(0,n):
        #create cos values
        cosval = np.cos((2*math.pi/period)*i * x_val)
        #create cos * dx
        coef = np.multiply(cosval,dx)
        #integrate data values
        out[i] = np.dot(coef,data)

    #rescale
    out = math.sqrt(2)*out/(period)
    return out

In [52]:
def plot_song(song):
    hop_length = 1024
    n_fft = 2048
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    DB = librosa.amplitude_to_db(D, ref=np.max)
    librosa.display.specshow(DB, hop_length=hop_length, x_axis='time', y_axis='log');
    plt.colorbar(format='%+2.0f dB');
    plt.show()
    return None

def plot_mel(S):
    fig, ax = plt.subplots()
    S_dB = librosa.power_to_db(S, ref=np.max)
    img = librosa.display.specshow(S_dB, x_axis='time',
                             y_axis='mel')
    fig.colorbar(img, ax=ax, format='%+2.0f dB')
    ax.set(title='Mel-frequency spectrogram')
    plt.show()
    return None

In [72]:
def mfcc_by_hand(S, mel_spec = False, freqMin = 0, n = 20):
    numFreq = S.shape[0]
    freqMax = freqMin + numFreq
    time = S.shape[1]
    
    #input spectogram or mel spectogram
    if mel_spec: #mel spectogram
        DB = librosa.power_to_db(S, ref=np.max)
    else: #normal spectogram
        DB = librosa.amplitude_to_db(S, ref=np.max)
        
    #compute mel scale freq index
    freq = np.zeros(numFreq, dtype=float, order='C')
    for i in range(0,numFreq):
        freq[i] = i+freqMin
    #convert to melscale
    mfreq = 2595*np.log10(1+freq/700+1/1400) #the 1400 is because in DCT you pretend x_values are at half integers
    period = 2*(2595*np.log10(1+freqMax/700) - 2595*np.log10(1+freqMin/700))
    print (period)
    
    #apply adjusted dct
    out = np.zeros((n,time), dtype=float, order='C')
    for i in range(0,time):
        col = DB[:,i]
        out[:,i] = adjusted_dct(col, mfreq, period, n=n)
    return out
    

In [79]:
def get_hand_mfcc(directory, print_filename = False, n=20):
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            if print_filename:
                print (filename)
            song, sr = librosa.load(f)
            D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
            out = mfcc_by_hand(D, mel_spec = False, freqMin = 0, n = n)
            print (out[0:5,0:5])
            print (mfcc(song,n=n)[0:5,0:5])
    return None

def get_melspec(directory, print_filename=False):
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            if print_filename:
                print (filename)
            song, sr = librosa.load(f)
            out = mel_specto(song)
            plot_mel(out)
    return None

In [9]:
directory = 'Small_audio_sample'
hop_length = 1024
n_fft = 2048

In [82]:
get_hand_mfcc(directory, print_filename=True, n=5)

122219.wav
2032.8765982602374
[[-44.16345997 -34.35655316 -39.78574977 -41.32043006 -39.42789828]
 [  6.48613402   7.54679867   7.14011112   5.80768548   6.68966335]
 [  0.95519886   0.75569549   1.50693082   2.28156042   1.11866348]
 [  0.37019066  -0.48813399  -0.17633015   1.13420617  -0.88711014]
 [  0.22782236  -0.7094348   -0.46668562  -0.89131359  -0.13745887]]
[[-3.1139230e+02 -1.7617947e+02 -1.3333643e+02 -1.6280728e+02
  -2.0693887e+02]
 [ 1.5062582e+02  1.6224936e+02  1.5496732e+02  1.5670787e+02
   1.7413788e+02]
 [-2.6672890e+01 -5.2640747e+01 -6.0652000e+01 -6.2892067e+01
  -4.3138542e+01]
 [ 2.1329426e+01  1.5504133e+01 -6.2390566e-03 -8.4361477e+00
  -1.7131004e+01]
 [-2.3500476e+00  7.6207399e-01 -5.6126013e+00 -1.0585268e+01
  -1.5461532e+01]]
119884.wav
2032.8765982602374
[[-5.53248535e+01 -5.28763329e+01 -4.49053073e+01 -4.69540380e+01
  -4.83186294e+01]
 [ 1.11251166e+00  2.92599492e+00  7.40238887e+00  6.55196449e+00
   5.72332323e+00]
 [ 8.17712478e-01  1.7101874